In [1]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.optim as optimizers
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

np.random.seed(9837)
torch.manual_seed(9837)
pd.set_option("display.max_rows", 250)
pd.set_option("display.max_columns", 100)

In [2]:
# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

# ディリクリ分布の乱数を生成する関数
def Dirichlet(alpha, n):
    x = torch.Tensor(np.random.dirichlet(alpha, n))
    return x

# データの生成

## 入力データの定義

In [5]:
# データの設定
types = 2
min_word = 5
max_word = 50
k11 = 5   # topic wordのsyntax数
k12 = 7   # general wordのsyntax数
k1 = k11 + k12   # syntax数
k21 = 15   # topic wordのトピック数
k22 = 10   # general wordのトピック数
k2 = k21 + k22   # topic数
k3 = 15   # word classのトピック数
d = 5000   # 文書数
v11 = 1000  # topic wordのvocabulary数
v12 = 400   # general wordのvocabulary数
v1 = v11 + v12   # vocabulary数
v2 = 100   # word class数
pt = np.random.poisson(np.random.gamma(12.5, 1.0, d), d)
pt[pt < 5] = 5
M = np.sum(pt)
w = np.random.poisson(np.random.gamma(17.5, 1.5, np.sum(pt)), np.sum(pt))
w[w < min_word] = min_word
N = np.sum(w)

# データベクトルを定義
k_vec1 = np.repeat(1.0, k1)
k_vec2 = np.repeat(1.0, k2)
k_vec21 = np.repeat(1.0, k21)
k_vec22 = np.repeat(1.0, k22)
index_k11 = np.arange(k11)
index_k12 = np.arange(k12) + k11
index_k21 = np.arange(k21)
index_k22 = np.arange(k22) + k21
index_v11 = np.arange(v11)
index_v12 = np.arange(v12) + v11

In [6]:
# IDとインデックスを定義
# IDの定義
m = np.repeat(0, d)
doc_list = []
d_id = []
doc_id = np.repeat(np.arange(d), pt)
for i in range(d):
    doc_list.append(np.where(doc_id==i)[0].astype("int"))
    m[i] = np.sum(w[doc_list[i]])
    d_id.append(np.repeat(i, m[i]))
d_id = np.hstack((d_id))
sentence_id = np.repeat(np.arange(M), w)
pt_id = np.hstack(([np.arange(w[i]) for i in range(M)]))

# 文書のインデックスを定義
d_list = []
sentence_list = []
for i in range(d):
    d_list.append(np.where(d_id==i)[0].astype("int"))
for i in range(M):
    sentence_list.append(np.where(sentence_id==i)[0].astype("int"))
    
# 語順のインデックスを定義
max_pt = np.max(pt_id) + 1
pt_list = []
pt_n = np.repeat(0, max_pt)
for j in range(max_pt):
    pt_list.append(np.array(np.where(pt_id==j)[0], dtype="int"))
    pt_n[j] = pt_list[j].shape[0]

## パラメータと応答変数を生成

In [7]:
# 事前分布の定義
# HMMの事前分布を定義
alpha1 = np.repeat(1.0, k1)
alpha2 = np.append(np.repeat(0.5, k1), 5.0)

# トピック分布の事前分布
beta1 = np.append(np.repeat(0.2, k21), np.repeat(0.15, k22))
beta2 = np.append(np.repeat(0.1, k21), np.repeat(0.2, k22))

# 単語分布の事前分布
max_word = 30
gamma11 = np.full((k1, v11), 0.01)
gamma12 = np.full((k1, v12), 0.005)
gamma11[index_k11, ] = 0.05
gamma12[index_k12, ] = 0.05
gamma1 = np.hstack((gamma11, gamma12))
gamma21 = np.full((k2, v11), 0.0025)
gamma22 = np.full((k2, v12), 0.001)
gamma21[index_k21, ] = 0.025
gamma22[index_k22, ] = 0.025
gamma2 = np.hstack((gamma21, gamma22))

In [8]:
# パラメータを生成
# HMMの推移確率を生成
pi1 = np.append(np.random.dirichlet(alpha1, 1), 0.0).reshape(-1)
while True:
    pi2 = np.random.dirichlet(alpha2, k1+1)
    if (np.mean(pi2[:, k1]) > 0.45) & (np.mean(pi2[:, k1]) < 0.6):
        break
pit1 = pi1.copy(); pit2 = pi2.copy()

# ディリクリ分布からトピック分布を生成
kappa = np.random.normal(0, 0.75, v1)
theta = np.vstack((np.random.dirichlet(beta1, v11), 
                   np.random.dirichlet(beta2, v12)))
kappat = kappa.copy(); thetat = theta.copy()

# 単語分布の事前分布
psi = np.array([np.random.dirichlet(gamma1[j, ], 1).reshape(-1) for j in range(k1)])
phi = np.array([np.random.dirichlet(gamma2[j, ], 1).reshape(-1) for j in range(k2)])
psit = psi.copy(); phit = phi.copy()

In [9]:
# 応答変数を生成
# 生成したデータの格納用配列
S = np.zeros((N, k1+1), dtype="int")
s = np.repeat(0, N)
Z = np.zeros((N, k2), dtype="int")
z = np.repeat(-1, N)
word_id = np.repeat(0, N).astype("int16")
word_long = np.full((N, max_pt), -1, dtype="int16")
attention_id = np.repeat(-1, N).astype("int16")

# トピックと単語を生成
for j in range(max_pt):

    # 語順に応じた生成を実行
    if j==0:
        
        # 語順が先頭の単語を生成
        # 多項分布からsyntaxを生成
        index = pt_list[j]
        S[index, ] = np.random.multinomial(1, pi1, pt_n[j])
        s[index] = np.dot(S[index, ], np.arange(k1+1))

        # 単語を生成
        word_id[index] = rmnom(psi[s[index], ], pt_n[j], v1, 0)
        word_long[index, j] = word_id[index, ]
        
    else:
        
        # 語順が2単語目以降の単語を生成
        # 多項分布からsyntaxを生成
        index = pt_list[j]
        res = rmnom(pi2[s[index-1], ], pt_n[j], k1+1, 1)
        S[index, ] = res[1]
        s[index] = res[0]

        # 単語履歴を更新
        for q in range(j):
            word_long[index, q] = word_long[index-1, q]
        if j < max_word:
            index_col = np.arange(j)
        else:
            index_col = np.arange(j-max_word, j)

        # attentionの単語を選択
        index_hmm = index[np.array(np.where(res[1][:, k1]==0)[0], dtype="int")]
        index_attention = index[np.array(np.where(res[1][:, k1]==1)[0], dtype="int")]  
        m1 = index_hmm.shape[0]
        m2 = index_attention.shape[0]
        
        if m2 > 0:
            candidate_word = word_long[index_attention-1, ][:, index_col]
            logit = kappa[candidate_word, ]
            prob = np.exp(logit) / np.sum(np.exp(logit), axis=1)[:, np.newaxis]
            word = np.sum(candidate_word * rmnom(prob, m2, prob.shape[1], 1)[1], axis=1)
            attention_id[index_attention] = word

            # attentionからトピックを生成
            res = rmnom(theta[word, ], word.shape[0], k2, 1)
            Z[index_attention, ] = res[1]
            z[index_attention] = res[0]

        # 単語を生成
        word_id[index_hmm] = rmnom(psi[s[index_hmm], ], m1, v1, 0)
        word_id[index_attention] = rmnom(phi[z[index_attention], ], m2, v1, 0)
        word_long[index, j] = word_id[index]

In [45]:
# 一部データに教師をつける
supervised_prob = 0.25
index_y = np.where(s==k1)[0].astype("int")
y = np.repeat(0, N)
y[index_y] = np.random.binomial(1, 0.25, index_y.shape[0])
wd = word_id[index_y]

# attentionの単語集合をセット
attention_set = np.full((N, max_word), -1, dtype="int16")
for j in range(1, max_pt):
    index = pt_list[j]
    if j < max_word:
        index_col = np.arange(max_word)
    else:
        index_col = np.arange(j-max_word, j)
    attention_set[index, ] = word_long[index-1, ][:, index_col]
word_set = attention_set.copy()
word_set[y==1] = -1

# 単語集合のインデックスを定義
set_list = [j for j in range(max_word+1)]
set_id = [j for j in range(max_word+1)] 
set_list[0] = np.array([])
set_id[0] = np.array([]) 
for j in range(max_word):
    set_list[j+1] = np.array(np.where(word_set[:, j]!=-1)[0], dtype="int")
    set_id[j+1] = word_set[set_list[j+1], j]

# Semi-Supervised Self Attention LDAを推定

In [55]:
# アルゴリズムの設定
R = 2000
keep = 2
burnin = 500
skeep = int(burnin/keep)
iters = 0
disp = 10

In [56]:
# インデックスの設定
# 先頭と末尾のインデックスを作成
max_pt = np.max(pt_id) + 1
index_t11 = np.where(pt_id==0)[0].astype("int")
index_t12 = np.repeat(0, d)
for i in range(d):
    index_t12[i] = np.max(d_list[i])
    
# 中間のインデックスを作成
index_list_t21 = [j for j in range(max_pt-1)]
index_list_t22 = [j for j in range(max_pt-1)]
for j in range(1, max_pt):
    index_list_t21[j-1] = np.where(pt_id==j)[0].astype("int") - 1
    index_list_t22[j-1] = np.where(pt_id==j)[0].astype("int")
index_t21 = np.sort(np.array(list(itertools.chain(*[index_list_t21[j] for j in range(max_pt-1)]))))
index_t22 = np.sort(np.array(list(itertools.chain(*[index_list_t22[j] for j in range(max_pt-1)]))))

In [57]:
# パラメータの事前分布を定義
# HMMの事前分布を定義
alpha1 = np.repeat(1.0, k1)
alpha2 = np.append(np.repeat(0.5, k1), 5.0)

# トピック分布の事前分布
beta1 = 0.1
beta2 = 0.1

# 単語分布の事前分布
gamma1 = 0.05
gamma2 = 0.05

In [78]:
# パラメータの真値
# 推移確率とトピック分布の真値
pi1 = pit1.copy()
pi2 = pit2.copy()
kappa = kappat.copy()
theta = thetat.copy()

# 単語分布の真値
psi1 = psit[:, index_v11] / np.sum(psit[:, index_v11], axis=1)[:, np.newaxis]
psi2 = psit[:, index_v12] / np.sum(psit[:, index_v12], axis=1)[:, np.newaxis]
psi = np.hstack((psi1, psi2))
phi1 = phit[:, index_v11] / np.sum(phit[:, index_v11], axis=1)[:, np.newaxis]
phi2 = phit[:, index_v12] / np.sum(phit[:, index_v12], axis=1)[:, np.newaxis]
phi = np.hstack((phi1, phi2))

# 潜在変数の真値
Si = S.copy()
Zi = Z.copy()
s = np.dot(Si, np.arange(k1+1))
z = np.dot(Zi, np.arange(k2))

In [70]:
# パラメータの初期値
# 推移確率とトピック分布の初期値
pi1 = np.append(np.random.dirichlet(alpha1, 1), 0.0).reshape(-1)
pi2 = np.random.dirichlet(alpha2, k1+1)
kappa = np.random.normal(0, 0.75, v1)
theta = np.vstack((np.random.dirichlet(np.repeat(beta1, k2), v11), 
                   np.random.dirichlet(np.repeat(beta2, k2), v12)))

# 単語分布の初期値
psi1 = np.random.dirichlet(np.repeat(1.0, v11), k1)
psi2 = np.random.dirichlet(np.repeat(1.0, v12), k1)
psi = np.hstack((psi1, psi2))
phi1 = np.random.dirichlet(np.repeat(1.0, v11), k2)
phi2 = np.random.dirichlet(np.repeat(1.0, v12), k2)
phi = np.hstack((psi1, psi2))

# 潜在変数の初期値
Si = np.random.multinomial(1, np.random.dirichlet(np.repeat(1.0, k1+1), 1).reshape(-1), N)
Zi = np.random.multinomial(1, np.random.dirichlet(np.repeat(1.0, k2), 1).reshape(-1), N)
Si[index_y, ] = 0
Si[index_y, k1] = 1
s = np.dot(Si, np.arange(k1+1))
z = np.dot(Zi, np.arange(k2))

## パラメータをサンプリング

In [79]:
# 事後分布を定義
# syntaxとトピックの尤度を定義
phi_long = (phi.T)[word_id, ]
Lho1 = (psi.T)[word_id, ]   # syntaxごとの尤度

In [72]:
for j in range(1, max_word+1):
    theta[set_id[j], ] * phi_long[set_list[j], ]

In [84]:
j = 1
np.dot(theta[set_id[j], ] * phi_long[set_list[j], ], k_vec2)

array([0.00080195, 0.00399579, 0.00102145, ..., 0.00428589, 0.00989484,
       0.01790802])

In [83]:
set_list[j]

array([      1,       2,       4, ..., 1642451, 1642452, 1642453])